<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/llm/llama_2_llama_cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# LlamaCPP 

在这个简短的笔记本中，我们将展示如何使用 [llama-cpp-python](https://github.com/abetlen/llama-cpp-python) 库与 LlamaIndex。

在这个笔记本中，我们将使用 [`llama-2-chat-13b-ggml`](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML) 模型，以及适当的提示格式。

请注意，如果您使用的是 `llama-cpp-python` 版本 `0.1.79` 之后的版本，模型格式已从 `ggmlv3` 更改为 `gguf`。像本笔记本中使用的旧模型文件可以使用 [`llama.cpp`](https://github.com/ggerganov/llama.cpp) 仓库中的脚本进行转换。或者，您可以从 [huggingface](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF) 下载上述模型的 GGUF 版本。

默认情况下，如果 model_path 和 model_url 为空，`LlamaCPP` 模块将根据您的版本以任一格式加载 llama2-chat-13B。

## 安装

为了获得 `LlamaCPP` 的最佳性能，建议安装该软件包，以便使用 GPU 支持进行编译。安装方式的完整指南在[这里](https://github.com/abetlen/llama-cpp-python#installation-with-openblas--cublas--clblast--metal)。

完整的 MACOS 指令也在[这里](https://llama-cpp-python.readthedocs.io/en/latest/install/macos/)。

一般而言：
- 如果您有 CUDA 和 NVIDIA GPU，请使用 `CuBLAS`
- 如果您在 M1/M2 MacBook 上运行，请使用 `METAL`
- 如果您在 AMD/Intel GPU 上运行，请使用 `CLBLAST`


In [ ]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

## 设置LLM

LlamaCPP llm 高度可配置。根据所使用的模型，您将需要传入 `messages_to_prompt` 和 `completion_to_prompt` 函数来帮助格式化模型输入。

由于默认模型是 llama2-chat，我们使用在 [`llama_index.llms.llama_utils`](https://github.com/jerryjliu/llama_index/blob/main/llama_index/llms/llama_utils.py) 中找到的实用函数。

对于任何需要在初始化期间传入的 kwargs，请将它们设置在 `model_kwargs` 中。可用模型 kwargs 的完整列表可在 [LlamaCPP 文档](https://llama-cpp-python.readthedocs.io/en/latest/api-reference/#llama_cpp.llama.Llama.__init__) 中找到。

对于需要在推断期间传入的任何 kwargs，您可以将它们设置在 `generate_kwargs` 中。在这里查看完整的 [generate kwargs 列表](https://llama-cpp-python.readthedocs.io/en/latest/api-reference/#llama_cpp.llama.Llama.__call__)。

一般来说，默认值是一个很好的起点。下面的示例显示了所有默认配置。

如上所述，我们在本笔记本中使用的是 [`llama-2-chat-13b-ggml`](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML) 模型，它使用 `ggmlv3` 模型格式。如果您正在运行的 `llama-cpp-python` 版本大于 `0.1.79`，您可以将下面的 `model_url` 替换为 `"https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"`。


如果您在Colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
!pip install llama-index

In [ ]:
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"

In [ ]:
llm = LlamaCPP(    # 您可以传入GGML模型的URL以自动下载    model_url=model_url,    # 可选地，您可以设置预先下载的模型的路径，而不是model_url    model_path=None,    temperature=0.1,    max_new_tokens=256,    # llama2具有4096个标记的上下文窗口，但我们将其设置得较低以留有一些余地    context_window=3900,    # 传递给__call__()的kwargs    generate_kwargs={},    # 传递给__init__()的kwargs    # 设置为至少1以使用GPU    model_kwargs={"n_gpu_layers": 1},    # 将输入转换为Llama2格式    messages_to_prompt=messages_to_prompt,    completion_to_prompt=completion_to_prompt,    verbose=True,)

llama.cpp: loading model from /Users/rchan/Library/Caches/llama_index/models/llama-2-13b-chat.ggmlv3.q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 3900
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_internal: mem required  = 6983.72 MB (+ 3046.88 MB per state)
llama_new_context_wit

我们可以通过日志记录来判断模型正在使用`metal`！


## 开始使用我们的 `LlamaCPP` LLM 抽象！

我们可以简单地使用我们的 `LlamaCPP` LLM 抽象的 `complete` 方法来生成给定提示的补全。


In [ ]:
response = llm.complete("Hello! Can you tell me a poem about cats and dogs?")
print(response.text)

  Of course, I'd be happy to help! Here's a short poem about cats and dogs:

Cats and dogs, so different yet the same,
Both furry friends, with their own special game.

Cats purr and curl up tight,
Dogs wag their tails with delight.

Cats hunt mice with stealthy grace,
Dogs chase after balls with joyful pace.

But despite their differences, they share,
A love for play and a love so fair.

So here's to our feline and canine friends,
Both equally dear, and both equally grand.



llama_print_timings:        load time =  1204.19 ms
llama_print_timings:      sample time =   106.79 ms /   146 runs   (    0.73 ms per token,  1367.14 tokens per second)
llama_print_timings: prompt eval time =  1204.14 ms /    81 tokens (   14.87 ms per token,    67.27 tokens per second)
llama_print_timings:        eval time =  7468.88 ms /   145 runs   (   51.51 ms per token,    19.41 tokens per second)
llama_print_timings:       total time =  8993.90 ms


我们可以使用`stream_complete`端点来流式传输响应，而不是等待整个响应生成完毕。


In [ ]:
response_iter = llm.stream_complete("Can you write me a poem about fast cars?")
for response in response_iter:
    print(response.delta, end="", flush=True)

Llama.generate: prefix-match hit


  Sure! Here's a poem about fast cars:

Fast cars, sleek and strong
Racing down the highway all day long
Their engines purring smooth and sweet
As they speed through the streets

Their wheels grip the road with might
As they take off like a shot in flight
The wind rushes past with a roar
As they leave all else behind

With paint that shines like the sun
And lines that curve like a dream
They're a sight to behold, my son
These fast cars, so sleek and serene

So if you ever see one pass
Don't be afraid to give a cheer
For these machines of speed and grace
Are truly something to admire and revere.


llama_print_timings:        load time =  1204.19 ms
llama_print_timings:      sample time =   123.72 ms /   169 runs   (    0.73 ms per token,  1365.97 tokens per second)
llama_print_timings: prompt eval time =   267.03 ms /    14 tokens (   19.07 ms per token,    52.43 tokens per second)
llama_print_timings:        eval time =  8794.21 ms /   168 runs   (   52.35 ms per token,    19.10 tokens per second)
llama_print_timings:       total time =  9485.38 ms


## 使用LlamaCPP设置查询引擎

我们可以像往常一样，简单地将`LlamaCPP` LLM抽象传递给`LlamaIndex`查询引擎。

但首先，让我们将全局分词器更改为与我们的LLM匹配。


In [ ]:
from llama_index.core import set_global_tokenizer
from transformers import AutoTokenizer

set_global_tokenizer(
    AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf").encode
)

In [ ]:
# 使用Huggingface嵌入from llama_index.embeddings.huggingface import HuggingFaceEmbeddingembed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [ ]:
# 加载文档documents = SimpleDirectoryReader(    "../../../examples/paul_graham_essay/data").load_data()

In [ ]:
# 创建向量存储索引index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

In [ ]:
# 设置查询引擎query_engine = index.as_query_engine(llm=llm)

In [ ]:
response = query_engine.query("What did the author do growing up?")
print(response)

Llama.generate: prefix-match hit


  Based on the given context information, the author's childhood activities were writing short stories and programming. They wrote programs on punch cards using an early version of Fortran and later used a TRS-80 microcomputer to write simple games, a program to predict the height of model rockets, and a word processor that their father used to write at least one book.



llama_print_timings:        load time =  1204.19 ms
llama_print_timings:      sample time =    56.13 ms /    80 runs   (    0.70 ms per token,  1425.21 tokens per second)
llama_print_timings: prompt eval time = 65280.71 ms /  2272 tokens (   28.73 ms per token,    34.80 tokens per second)
llama_print_timings:        eval time =  6877.38 ms /    79 runs   (   87.06 ms per token,    11.49 tokens per second)
llama_print_timings:       total time = 72315.85 ms
